In [1]:
import os
import pandas as pd

In [2]:
import re
import json
import csv
from io import StringIO
from bs4 import BeautifulSoup
import requests

url_stats ='https://finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile ='https://finance.yahoo.com/quote/{}/profile?p={}'
url_financials ='https://finance.yahoo.com/quote/{}/financials?p={}'

In [3]:
stock = input()
response = requests.get(url_financials.format(stock, stock))

soup= BeautifulSoup(response.text, 'html.parser')

NIKL.JK


In [4]:
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]

In [5]:
# beginning
script_data[:500]

'\n(function (root) {\n/* -- Data -- */\nroot.App || (root.App = {});\nroot.App.now = 1607422091143;\nroot.App.main = {"context":{"dispatcher":{"stores":{"PageStore":{"currentPageName":"quote","currentEvent":{"eventName":"NEW_PAGE_SUCCESS"},"currentRenderTargetId":"default","pagesConfigRaw":{"base":{"quote":{"layout":{"bundleName":"yahoodotcom-layout.TwoColumnLayout","name":"TwoColumnLayout","config":{"enableHeaderCollapse":true,"additionalBodyWrapperClasses":"Bgc($layoutBgColor)!","contentWrapperClas'

In [6]:
# the end
script_data[-500:]

'":{"strings":1},"tdv2-applet-search-input":{"strings":1},"tdv2-applet-sponsored-moments":{"strings":1},"tdv2-applet-stream":{"strings":1},"tdv2-applet-stream-hero":{"strings":1},"tdv2-applet-swisschamp":{"strings":1},"tdv2-applet-uh":{"strings":1},"tdv2-applet-userintent":{"strings":1},"tdv2-applet-video-lightbox":{"strings":1},"tdv2-applet-video-modal":{"strings":1},"tdv2-wafer-header":{"strings":1},"yahoodotcom-layout":{"strings":1}}},"options":{"defaultBundle":"td-app-finance"}}}};\n}(this));\n'

In [7]:
# find the starting position of the json string
start = script_data.find("context")-2

# slice the json string
json_data = json.loads(script_data[start:-12])

# Financial statements
Now that you have the data, you can explore the dictionary to discover what's inside. This dataset contains both Annual and Quarterly financial statements, as you can see from the dictionary paths listed below.

In [8]:
json_data['context'].keys()

dict_keys(['dispatcher', 'options', 'plugins'])

In [9]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['financialsTemplate', 'cashflowStatementHistory', 'balanceSheetHistoryQuarterly', 'earnings', 'price', 'incomeStatementHistoryQuarterly', 'incomeStatementHistory', 'balanceSheetHistory', 'cashflowStatementHistoryQuarterly', 'quoteType', 'summaryDetail', 'symbol', 'pageViews'])

In [10]:
# income statement
annual_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
quarterly_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistoryQuarterly']['incomeStatementHistory']

# cash flow statement
annual_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements']
quarterly_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistoryQuarterly']['cashflowStatements']

# balance sheet
annual_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
quarterly_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistoryQuarterly']['balanceSheetStatements']

## Income Statements

In [11]:
# example of income statmement accounts
print(annual_is[0])

{'researchDevelopment': {}, 'effectOfAccountingCharges': {}, 'incomeBeforeTax': {'raw': 3896699, 'fmt': '3.9M', 'longFmt': '3,896,699'}, 'minorityInterest': {}, 'netIncome': {'raw': 2680666, 'fmt': '2.68M', 'longFmt': '2,680,666'}, 'sellingGeneralAdministrative': {'raw': 7158819, 'fmt': '7.16M', 'longFmt': '7,158,819'}, 'grossProfit': {'raw': 10467156, 'fmt': '10.47M', 'longFmt': '10,467,156'}, 'ebit': {'raw': 4410410, 'fmt': '4.41M', 'longFmt': '4,410,410'}, 'endDate': {'raw': 1577750400, 'fmt': '2019-12-31'}, 'operatingIncome': {'raw': 4410410, 'fmt': '4.41M', 'longFmt': '4,410,410'}, 'otherOperatingExpenses': {'raw': -1102073, 'fmt': '-1.1M', 'longFmt': '-1,102,073'}, 'interestExpense': {'raw': -1766713, 'fmt': '-1.77M', 'longFmt': '-1,766,713'}, 'extraordinaryItems': {}, 'nonRecurring': {}, 'otherItems': {}, 'incomeTaxExpense': {'raw': 1216033, 'fmt': '1.22M', 'longFmt': '1,216,033'}, 'totalRevenue': {'raw': 163085289, 'fmt': '163.09M', 'longFmt': '163,085,289'}, 'totalOperatingExp

0 --> 2019
1 --> 2018
2 --> 2017
3 --> 2016

In [12]:
# there's a variety of  number formats provided
annual_is[3]['operatingIncome']

{'raw': 2030937, 'fmt': '2.03M', 'longFmt': '2,030,937'}

In [13]:
annual_income_stmts = []

# consolidate annual
for s in annual_is:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_income_stmts.append(statement)
    
quarterly_income_stmts = []

# quarterly
for s in quarterly_is:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    quarterly_income_stmts.append(statement)

In [15]:
annual_income_stmts[0]

{'incomeBeforeTax': 3896699,
 'netIncome': 2680666,
 'sellingGeneralAdministrative': 7158819,
 'grossProfit': 10467156,
 'ebit': 4410410,
 'endDate': 1577750400,
 'operatingIncome': 4410410,
 'otherOperatingExpenses': -1102073,
 'interestExpense': -1766713,
 'incomeTaxExpense': 1216033,
 'totalRevenue': 163085289,
 'totalOperatingExpenses': 158674879,
 'costOfRevenue': 152618133,
 'totalOtherIncomeExpenseNet': -513711,
 'netIncomeFromContinuingOps': 2680666,
 'netIncomeApplicableToCommonShares': 2680666}

### IS Annual

In [17]:
annual_income_statement=pd.DataFrame(annual_income_stmts)
year=['2019','2018','2017','2016']
year=pd.DataFrame(year)
annual_income_statement['year']=year
annual_income_statement

,incomeBeforeTax,netIncome,sellingGeneralAdministrative,grossProfit,ebit,endDate,operatingIncome,otherOperatingExpenses,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,year
0,3896699,2680666,7158819,10467156,4410410,1577750400,4410410,-1102073,-1766713,1216033,163085289,158674879,152618133,-513711,2680666,2680666,2019
1,-1755985,-1537262,6589075,8318952,2065617,1546214400,2065617,-335740,-1791221,-218723,163135351,161069734,154816399,-3821602,-1537262,-1537262,2018
2,1441114,1359171,7913875,10118853,2550473,1514678400,2550473,-345495,-1792316,81943,151792945,149242472,141674092,-1109359,1359171,1359171,2017
3,2438754,2519314,9239397,10863659,2030937,1483142400,2030937,-406675,-581553,-80560,131664283,129633346,120800624,407817,2519314,2519314,2016


### IS Quarterly

In [18]:
quarterly_income_statement=pd.DataFrame(quarterly_income_stmts)
quarter=['Q3 2020','Q2 2020','Q1 2020','Q4 2019','Q3 2019']
quarter=pd.DataFrame(quarter)
quarterly_income_statement['quarter']=quarter
quarterly_income_statement

,incomeBeforeTax,netIncome,sellingGeneralAdministrative,grossProfit,ebit,endDate,operatingIncome,otherOperatingExpenses,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,quarter
0,-1538997,-1344288,2083696,2222086,339041,1601424000,339041,-200651,-136015,-194709,36289993,35950952,34067907,-1878038,-1344288,-1344288,Q3 2020
1,10624469,10430897,1075234,2038578,1146349,1593475200,1146349,-183005,-235136,193572,31719035,30572686,29680457,9478120,10430897,10430897,Q2 2020
2,-10319966,-10140844,2491092,1498988,-776620,1585612800,-776620,-215484,-326152,-179122,38018411,38795031,36519423,-9543346,-10140844,-10140844,Q1 2020
3,1146718,840195,1984946,2022934,280082,1577750400,280082,-242094,-353727,306523,39290888,39010806,37267954,866636,840195,840195,Q4 2019


This model can be applied to all other financial statements, as you can see from the examples below.

## Balance Sheets

In [19]:
annual_bs_stmts = []
quarterly_bs_stmts = []

# annual
for s in annual_bs:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_bs_stmts.append(statement)
    
# quarterly
for s in quarterly_bs:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    quarterly_bs_stmts.append(statement)

In [20]:
annual_bs_stmts[0]

{'capitalSurplus': 11413745,
 'totalLiab': 106049817,
 'totalStockholderEquity': 45639161,
 'otherCurrentLiab': 1207361,
 'totalAssets': 151688978,
 'endDate': 1577750400,
 'commonStock': 26844149,
 'otherCurrentAssets': 3011193,
 'retainedEarnings': -4586514,
 'otherLiab': 2610374,
 'treasuryStock': 11967781,
 'otherAssets': 4710169,
 'cash': 28392647,
 'totalCurrentLiabilities': 103439443,
 'otherStockholderEquity': 11967781,
 'propertyPlantEquipment': 29162421,
 'totalCurrentAssets': 117688728,
 'longTermInvestments': 127660,
 'netTangibleAssets': 45639161,
 'netReceivables': 56328338,
 'inventory': 29956550,
 'accountsPayable': 26311173}

### BS Annual

In [21]:
annual_bs_statement=pd.DataFrame(annual_bs_stmts)
year=['2019','2018','2017','2016']
year=pd.DataFrame(year)
annual_bs_statement['year']=year
annual_bs_statement

,capitalSurplus,totalLiab,totalStockholderEquity,otherCurrentLiab,totalAssets,endDate,commonStock,otherCurrentAssets,retainedEarnings,otherLiab,...,otherStockholderEquity,propertyPlantEquipment,totalCurrentAssets,longTermInvestments,netTangibleAssets,netReceivables,inventory,accountsPayable,deferredLongTermAssetCharges,year
0,11413745,106049817,45639161,1207361,151688978,1577750400,26844149,3011193,-4586514,2610374,...,11967781,29162421,117688728,127660,45639161,56328338,29956550,26311173,NaN,2019
1,11413745,104723676,43053536,937936,147777212,1546214400,26844149,4387156,-7172139,2164786,...,11967781,30270929,112805625,127660,43053536,57009414,40305442,31575798,1072351.0,2018
2,11413745,84476044,41646797,1049060,126122841,1514678400,26844149,381931,-6201215,4457658,...,9590118,25517665,94640013,127660,41646797,49701955,30047808,24259470,1042407.0,2017
3,11413745,79660396,40007396,1357239,119667792,1483142400,26844149,582754,-7840616,4671743,...,9590118,25962279,87751429,127660,40007396,38226737,27771729,24759010,1139994.0,2016


### BS Quarterly

In [22]:
quarterly_bs_statement=pd.DataFrame(quarterly_bs_stmts)
quarter=['Q3 2020','Q2 2020','Q1 2020','Q4 2019','Q3 2019']
quarter=pd.DataFrame(quarter)
quarterly_bs_statement['quarter']=quarter
quarterly_bs_statement

,capitalSurplus,totalLiab,totalStockholderEquity,otherCurrentLiab,totalAssets,endDate,commonStock,otherCurrentAssets,retainedEarnings,otherLiab,...,deferredLongTermAssetCharges,otherStockholderEquity,propertyPlantEquipment,totalCurrentAssets,longTermInvestments,netTangibleAssets,netReceivables,inventory,accountsPayable,quarter
0,11413745,79640950,44546461,1148516,124187411,1601424000,26844149,4120313,-5679214,2055580,...,58010.0,11967781,28279144,94298586,127660,44546461,52053576,23336623,16137436,Q3 2020
1,11413745,91328109,45878240,1158905,137206349,1593475200,26844149,4352716,-4347435,2305007,...,937.0,11967781,28693967,106947732,127660,45878240,49112602,30172764,23572318,Q2 2020
2,11413745,107257698,35360807,1136349,142618505,1585612800,26844149,3451125,-14864868,2188097,...,89887.0,11967781,29250821,107820473,127660,35360807,47598434,26012326,24665046,Q1 2020
3,11413745,106049817,45639161,1207361,151688978,1577750400,26844149,3011193,-4586514,2610374,...,NaN,11967781,29162421,117688728,127660,45639161,56328338,29956550,26311173,Q4 2019


## Cash Flow Statements

In [23]:
annual_cf_stmts = []
quarterly_cf_stmts = []

# annual
for s in annual_cf:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_cf_stmts.append(statement)
    
# quarterly
for s in quarterly_cf:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    quarterly_cf_stmts.append(statement)

In [24]:
annual_cf_stmts[0]

{'changeToLiabilities': 0,
 'totalCashflowsFromInvestingActivities': -1623186,
 'netBorrowings': 8404505,
 'totalCashFromFinancingActivities': 8404505,
 'netIncome': 2680666,
 'changeInCash': 17289034,
 'endDate': 1577750400,
 'effectOfExchangeRate': 329756,
 'totalCashFromOperatingActivities': 10177959,
 'depreciation': 2441492,
 'changeToNetincome': 5055801,
 'capitalExpenditures': -1640462}

### CF Annual

In [25]:
annual_cf_statement=pd.DataFrame(annual_cf_stmts)
year=['2019','2018','2017','2016']
year=pd.DataFrame(year)
annual_cf_statement['year']=year
annual_cf_statement

,changeToLiabilities,totalCashflowsFromInvestingActivities,netBorrowings,totalCashFromFinancingActivities,netIncome,changeInCash,endDate,effectOfExchangeRate,totalCashFromOperatingActivities,depreciation,changeToNetincome,capitalExpenditures,year
0,0,-1623186,8404505,8404505,2680666,17289034,1577750400,329756,10177959,2441492,5055801,-1640462,2019
1,0,-4733120,12704899,12704899,-1537262,-3399235,1546214400,-488615,-10882399,2196502,-11541639,-4733120,2018
2,0,-1742324,6430268,6430268,1359171,-6610771,1514678400,-55988,-11242727,2343569,-14945467,-1754586,2017
3,0,-358469,-954563,-954563,2519314,12535291,1483142400,-55988,13848323,2937429,8391580,-358469,2016


### CF Quarterly

In [26]:
quarterly_cf_statement=pd.DataFrame(quarterly_cf_stmts)
quarter=['Q3 2020','Q2 2020','Q1 2020','Q4 2019','Q3 2019']
quarter=pd.DataFrame(quarter)
quarterly_cf_statement['quarter']=quarter
quarterly_cf_statement

,changeToLiabilities,totalCashflowsFromInvestingActivities,netBorrowings,totalCashFromFinancingActivities,netIncome,changeInCash,endDate,effectOfExchangeRate,totalCashFromOperatingActivities,depreciation,changeToNetincome,capitalExpenditures,quarter
0,0,-87535,-5013586,-5013586,-1344288,-8521576,1601424000,-775594,-2644861,602961,-1903534,-87535,Q3 2020
1,0,-50069,-15851874,-15851874,10430897,-7448938,1593475200,3017131,5435874,604313,-5599336,-50069,Q2 2020
2,0,-32434,2986038,2986038,-10140844,2365941,1585612800,-2135401,1547738,610445,11078137,-32434,Q1 2020
3,0,-931832,13508499,13508499,840195,17519028,1577750400,1339532,3602829,613541,2149093,-928861,Q4 2019


## Profile Data

In [27]:
response = requests.get(url_profile.format(stock, stock))
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])

In [28]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['financialsTemplate', 'price', 'secFilings', 'quoteType', 'calendarEvents', 'summaryDetail', 'symbol', 'assetProfile', 'pageViews'])

In [29]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile'].keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'fax', 'industry', 'address2'])

In [30]:
# data for company officers (just the first 3 are listed for brevity )
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['companyOfficers'][:3]

[{'maxAge': 1,
  'name': 'Mr. -  Jetrinaldi',
  'age': 51,
  'title': 'Pres Director',
  'yearBorn': 1968,
  'exercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'},
  'unexercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'}},
 {'maxAge': 1,
  'name': 'Mr. Kazumi  Okamoto',
  'title': 'Operation Director & VP Director',
  'exercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'},
  'unexercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'}},
 {'maxAge': 1,
  'name': 'Mr. Alfa  Enersi',
  'age': 49,
  'title': 'Fin. Director & Director',
  'yearBorn': 1970,
  'exercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'},
  'unexercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'}}]

In [31]:
# business description
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['longBusinessSummary']

'PT Pelat Timah Nusantara Tbk operates as a tinplate manufacturer in Indonesia and internationally. It offers tinplate products in the form of coils, and sheets and scrolls. The company provides tinplate for consumer products, such as food and beverages, canned milk, battery, paints, and chemicals industries. PT Pelat Timah Nusantara Tbk was founded in 1982 and is headquartered in Jakarta, Indonesia.'

In [32]:
# lot of other data is available
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['summaryDetail']

{'previousClose': {'raw': 1355, 'fmt': '1,355.00'},
 'regularMarketOpen': {'raw': 1420, 'fmt': '1,420.00'},
 'twoHundredDayAverage': {'raw': 722.4857, 'fmt': '722.49'},
 'trailingAnnualDividendYield': {},
 'payoutRatio': {'raw': 0, 'fmt': '0.00%'},
 'volume24Hr': {},
 'regularMarketDayHigh': {'raw': 1690, 'fmt': '1,690.00'},
 'navPrice': {},
 'averageDailyVolume10Day': {'raw': 10528050,
  'fmt': '10.53M',
  'longFmt': '10,528,050'},
 'totalAssets': {},
 'regularMarketPreviousClose': {'raw': 1355, 'fmt': '1,355.00'},
 'fiftyDayAverage': {'raw': 877.35297, 'fmt': '877.35'},
 'trailingAnnualDividendRate': {},
 'open': {'raw': 1420, 'fmt': '1,420.00'},
 'toCurrency': None,
 'averageVolume10days': {'raw': 10528050,
  'fmt': '10.53M',
  'longFmt': '10,528,050'},
 'expireDate': {},
 'yield': {},
 'algorithm': None,
 'dividendRate': {},
 'exDividendDate': {'raw': 1398816000, 'fmt': '2014-04-30'},
 'beta': {'raw': 1.116118, 'fmt': '1.12'},
 'circulatingSupply': {},
 'startDate': {},
 'regularMa

In [33]:
stock_data=pd.DataFrame(json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['summaryDetail']).reset_index()
stock_data=stock_data.loc[stock_data['index']=='fmt']
stock_data.dropna(axis=1,inplace=True)
stock_data.drop(columns=['index','priceHint','currency','maxAge','tradeable'],inplace=True)

# Statistics

In [34]:
response = requests.get(url_stats.format(stock, stock))
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])

In [35]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['defaultKeyStatistics']

{'annualHoldingsTurnover': {},
 'enterpriseToRevenue': {'raw': 23528.918, 'fmt': '23,528.92'},
 'beta3Year': {},
 'profitMargins': {'raw': -0.00147, 'fmt': '-0.15%'},
 'enterpriseToEbitda': {'raw': 999728.4, 'fmt': '999,728.38'},
 '52WeekChange': {'raw': 1.9807692, 'fmt': '198.08%'},
 'morningStarRiskRating': {},
 'forwardEps': {},
 'revenueQuarterlyGrowth': {},
 'sharesOutstanding': {'raw': 2523350016,
  'fmt': '2.52B',
  'longFmt': '2,523,350,016'},
 'fundInceptionDate': {},
 'annualReportExpenseRatio': {},
 'totalAssets': {},
 'bookValue': {'raw': 0.018, 'fmt': '0.02'},
 'sharesShort': {},
 'sharesPercentSharesOut': {},
 'fundFamily': None,
 'lastFiscalYearEnd': {'raw': 1577750400, 'fmt': '2019-12-31'},
 'heldPercentInstitutions': {'raw': 0, 'fmt': '0.00%'},
 'netIncomeToCommon': {'raw': -214040,
  'fmt': '-214.04k',
  'longFmt': '-214,040'},
 'trailingEps': {'raw': 0, 'fmt': '-0.00'},
 'lastDividendValue': {'raw': 0.48, 'fmt': '0.48'},
 'SandP52WeekChange': {'raw': 0.17859101, 'fmt

In [36]:
Stats=pd.DataFrame(json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['defaultKeyStatistics']).reset_index()
Stats=Stats.loc[Stats['index']=='fmt']
Stats.dropna(axis=1,inplace=True)
Stats.drop(columns=['index','priceHint','maxAge'],inplace=True)

# Historical Stock Data

In [37]:
stock=input()

NIKL.JK


In [38]:
stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/NIKL.JK?period1=1568483641&period2=1600106041&interval=1d&events=history'

In [39]:
response = requests.get(stock_url)

In [40]:
# extract the csv data
file = StringIO(response.text)
reader = csv.reader(file)
data = list(reader)

# show the first 5 records
for row in data[:5]:
    print(row)

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
['2019-09-16', '1305.000000', '1305.000000', '1250.000000', '1255.000000', '1255.000000', '1098300']
['2019-09-17', '1255.000000', '1275.000000', '1250.000000', '1265.000000', '1265.000000', '330100']
['2019-09-18', '1275.000000', '1280.000000', '1260.000000', '1260.000000', '1260.000000', '337800']
['2019-09-19', '1260.000000', '1280.000000', '1250.000000', '1250.000000', '1250.000000', '587000']


In [41]:
stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'

params = {
    'period1':'1568483641',
    'period2':'1600106041',
    'interval':'1d',
    'events':'history'
}

By inspecting the request headers and parameters online, it's possible to see how this can be simplified further... by using the range parameter instead of the periods.

In [42]:
params = {
    'range': '5y',
    'interval':'1d',
    'events':'history'
}

In [43]:
response = requests.get(stock_url.format(stock), params=params)

In [44]:
# extract the csv data
file = StringIO(response.text)
reader = csv.reader(file)
data = list(reader)

# show the first 5 records
for row in data[:100]:
    print(row)
    
   

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
['2015-12-08', '50.000000', '50.000000', '50.000000', '50.000000', '50.000000', '11200']
['2015-12-09', '50.000000', '50.000000', '50.000000', '50.000000', '50.000000', '0']
['2015-12-10', '50.000000', '50.000000', '50.000000', '50.000000', '50.000000', '1500']
['2015-12-11', '50.000000', '50.000000', '50.000000', '50.000000', '50.000000', '200']
['2015-12-14', '50.000000', '50.000000', '50.000000', '50.000000', '50.000000', '0']
['2015-12-15', '50.000000', '50.000000', '50.000000', '50.000000', '50.000000', '0']
['2015-12-16', '50.000000', '50.000000', '50.000000', '50.000000', '50.000000', '0']
['2015-12-17', '50.000000', '50.000000', '50.000000', '50.000000', '50.000000', '200']
['2015-12-18', '50.000000', '50.000000', '50.000000', '50.000000', '50.000000', '0']
['2015-12-21', '50.000000', '50.000000', '50.000000', '50.000000', '50.000000', '100']
['2015-12-22', '50.000000', '50.000000', '50.000000', '50.000000', '50.00

In [45]:
price_history=pd.DataFrame(data)
new_header = price_history.iloc[0] #grab the first row for the header
price_history = price_history[1:] #take the data less the header row
price_history.columns = new_header #set the header row as the price_history header
price_history

,Date,Open,High,Low,Close,Adj Close,Volume
1,2015-12-08,50.000000,50.000000,50.000000,50.000000,50.000000,11200
2,2015-12-09,50.000000,50.000000,50.000000,50.000000,50.000000,0
3,2015-12-10,50.000000,50.000000,50.000000,50.000000,50.000000,1500
4,2015-12-11,50.000000,50.000000,50.000000,50.000000,50.000000,200
5,2015-12-14,50.000000,50.000000,50.000000,50.000000,50.000000,0
...,...,...,...,...,...,...,...
1259,2020-12-02,1020.000000,1080.000000,1015.000000,1070.000000,1070.000000,13896100
1260,2020-12-03,1070.000000,1140.000000,1060.000000,1095.000000,1095.000000,18923600
1261,2020-12-04,1105.000000,1115.000000,1075.000000,1085.000000,1085.000000,7304500
1262,2020-12-07,1100.000000,1355.000000,1100.000000,1355.000000,1355.000000,39473900


# Compile to 1 Excel Sheet

In [46]:
stock_data=stock_data.T.reset_index()
stock_data.columns=['indicator','last']
stock_data

,indicator,last
0,previousClose,"1,355.00"
1,regularMarketOpen,"1,420.00"
2,twoHundredDayAverage,722.49
3,payoutRatio,0.00%
4,regularMarketDayHigh,"1,690.00"
5,averageDailyVolume10Day,10.53M
6,regularMarketPreviousClose,"1,355.00"
7,fiftyDayAverage,877.35
8,open,"1,420.00"
9,averageVolume10days,10.53M


In [47]:
Stats=Stats.T.reset_index()
Stats.columns=['indicator','last']
Stats

,indicator,last
0,enterpriseToRevenue,"23,528.92"
1,profitMargins,-0.15%
2,enterpriseToEbitda,"999,728.38"
3,52WeekChange,198.08%
4,sharesOutstanding,2.52B
5,bookValue,0.02
6,lastFiscalYearEnd,2019-12-31
7,heldPercentInstitutions,0.00%
8,netIncomeToCommon,-214.04k
9,trailingEps,-0.00


In [48]:
#change columns to numeric
cols = ['Open','High','Low','Close','Adj Close','Volume']
price_history[cols] = price_history[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [50]:
# function
def dfs_tabs(df_list, sheet_list, file_name):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save()

# list of dataframes and sheet names
dfs = [annual_income_statement,annual_bs_statement,annual_cf_statement,
       quarterly_income_statement,quarterly_bs_statement, quarterly_bs_statement,
       stock_data,Stats, price_history]
sheets = ['IS-Annual','BS-Annual','CF-Annual','IS-Quarterly','BS-Quarterly','CF-Quarterly',
          'Stock Data','Stats','Stock Price History']    

In [ ]:
stock=input()

In [51]:
# run function
dfs_tabs(dfs, sheets, 'NIKL Stock Analysis.xlsx')